In [ ]:


!pip install datasets transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 480.6/480.6 kB 8.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 179.3/179.3 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 19.4 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.10.0
    Uninstalling fsspec-2024.10.0:
      Successfully uninstalled fsspec-2024.10.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gcsfs 2024.10.0 requires fsspec==2024.10.0, but you have fsspec 2024.9.0 which is incompatible.


In [ ]:


import pandas as pd
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset
from google.colab import drive

In [ ]:

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:


df = pd.read_excel("/content/drive/MyDrive/cleaned_mental_health_dataset.xlsx")
df = df.dropna().reset_index(drop=True)


dataset = Dataset.from_pandas(df)

# Split  dataset
train_data, test_data = dataset.train_test_split(test_size=0.2).values()

# Checks to make sure my dataset isn't messed up.
print(f"Training data size: {len(train_data)}")
print(f"Testing data size: {len(test_data)}")


print("First example in the training data:", train_data[0])
print("First example in the testing data:", test_data[0])

Training data size: 20998
Testing data size: 5250
First example in the training data: {'statement': 'لدي الحب وما يكفي من المصاري لاعيشه وسقف فوق راسي وما توجد صدمه بتكلم عنها التقيت بتسلسل ماسلو الهرمي بس انا لسا اغزو وبحط مسدسا عراسي وبضغط عالزناد كل يوم هل الانتحار تحقيق للذات القيام بعمل منيح عالورق', 'label': 'Suicidal'}
First example in the testing data: {'statement': 'لا اعرف السبب  لانه لم يزعجني قبل ان اراه علي الشاشه  لكنني محطم تماما  لقد طلبت منه اخفاء الامر عني من الان فصاعدا واعربت عن مدي الالم الذي سببته  لكنني ذهبت لاستخدام نظام الالعاب الخاص به منذ بضعه ايام ورايت انه لا يزال يبحث عنه بانتظام  اعتقد انه اعتقد انني لن افعل ذلك  لا تستخدمه - لقد ذهبت للبحث عن شيء ما وظهرت جميع عمليات البحث في شريط Bing   لم اشعر ابدا بالجاذبيه تجاهه واعلم ان هذا خطءي وانعدام الامان وليس مشكلته التي يجب اصلاحها  اعلم بعقلانيه ان الرجال يحبون التنوع  لكنني لم ارسل له صورا عاريه منذ شهرين علي الارجح ', 'label': 'Stress'}


In [ ]:


print(f"Dataset size: {len(dataset)}")
print(dataset[0])  # Print the first example if the dataset isn't empty

Dataset size: 26248
{'statement': 'هل فهم ما هي المشكله ', 'label': 'Normal'}


In [ ]:


model_name = "UBC-NLP/MARBERT"
tokenizer = AutoTokenizer.from_pretrained(model_name)

# Tokenize
def tokenize_function(example):
    return tokenizer(example["statement"], padding="max_length", truncation=True, max_length=512)

train_data = train_data.map(tokenize_function, batched=True)
test_data = test_data.map(tokenize_function, batched=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/376 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/701 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.10M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

Map:   0%|          | 0/20998 [00:00<?, ? examples/s]

Map:   0%|          | 0/5250 [00:00<?, ? examples/s]

In [ ]:

post_lengths = [len(tokenizer(post["statement"])["input_ids"]) for post in dataset]


import numpy as np
mean_length = np.mean(post_lengths)
median_length = np.median(post_lengths)
max_length = np.max(post_lengths)
percentile_95 = np.percentile(post_lengths, 95)  # 95th percentile

print(f"Mean Length: {mean_length}")
print(f"Median Length: {median_length}")
print(f"Max Length: {max_length}")
print(f"95th Percentile Length: {percentile_95}")

Mean Length: 93.68595702529717
Median Length: 45.0
Max Length: 3609
95th Percentile Length: 332.0


In [ ]:
# Standardizer
df['label'] = df['label'].str.strip().str.capitalize()


print(df['label'].unique())

print(df.head())
print(df.columns)

['Normal' 'Anxiety' 'Depression' 'Stress' 'Suicidal']
                                           statement       label
0                              هل فهم ما هي المشكله       Normal
1  الخيام بتهتز مع كل هبه ريح  الشتاء هذا بيكون ك...     Anxiety
2   بيتنا اللي تعبنا تعبنا عشان حطام  صار نبنيه معه   Depression
3  سيكون من الجيد مشاركه وجهه النظر او معرفه كيف ...  Depression
4  لم احاول لهم  لا احب فكره الحاجه الي الحبوب لا...  Depression
Index(['statement', 'label'], dtype='object')


In [ ]:


label_mapping = {
    'Normal': 0,
    'Anxiety': 1,
    'Depression': 2,
    'Suicidal': 3,
    'Stress': 4
}


def map_labels(example):
    example['label'] = label_mapping.get(example['label'], -1)  # -1 for any invalid labels
    return example

# Apply mapping
train_data = train_data.map(map_labels)
test_data = test_data.map(map_labels)


train_data = train_data.filter(lambda x: x['label'] != -1)
test_data = test_data.filter(lambda x: x['label'] != -1)

# Convert  labels to integer format
train_data = train_data.map(lambda x: {"label": int(x["label"])})
test_data = test_data.map(lambda x: {"label": int(x["label"])})

# Set the correct format for the datases
train_data.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
test_data.set_format(type="torch", columns=["input_ids", "attention_mask", "label"])
print(set(train_data['label']))
print(set(test_data['label']))
print(f"Number of training examples: {len(train_data)}")
print(f"Number of testing examples: {len(test_data)}")

Map:   0%|          | 0/20998 [00:00<?, ? examples/s]

Map:   0%|          | 0/5250 [00:00<?, ? examples/s]

Filter:   0%|          | 0/20998 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5250 [00:00<?, ? examples/s]

Map:   0%|          | 0/20772 [00:00<?, ? examples/s]

Map:   0%|          | 0/5198 [00:00<?, ? examples/s]

{tensor(0), tensor(1), tensor(0), tensor(0), tensor(2), tensor(2), tensor(0), tensor(2), tensor(0), tensor(1), tensor(4), tensor(4), tensor(2), tensor(3), tensor(4), tensor(1), tensor(0), tensor(2), tensor(2), tensor(4), tensor(0), tensor(4), tensor(3), tensor(1), tensor(2), tensor(4), tensor(4), tensor(0), tensor(4), tensor(2), tensor(2), tensor(4), tensor(4), tensor(0), tensor(4), tensor(4), tensor(1), tensor(0), tensor(1), tensor(0), tensor(2), tensor(3), tensor(1), tensor(2), tensor(4), tensor(4), tensor(0), tensor(0), tensor(0), tensor(2), tensor(2), tensor(0), tensor(1), tensor(0), tensor(3), tensor(3), tensor(3), tensor(0), tensor(1), tensor(3), tensor(2), tensor(3), tensor(2), tensor(1), tensor(3), tensor(1), tensor(0), tensor(3), tensor(1), tensor(3), tensor(0), tensor(2), tensor(3), tensor(1), tensor(2), tensor(3), tensor(1), tensor(2), tensor(2), tensor(3), tensor(3), tensor(0), tensor(4), tensor(3), tensor(4), tensor(3), tensor(1), tensor(4), tensor(2), tensor(4), tensor(2)

In [ ]:

# Load MARBERT model with 5 labels
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=5)

# Define a function to compute evaluation metrics

pytorch_model.bin:   0%|          | 0.00/654M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at UBC-NLP/MARBERT and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:

from torch.utils.data import DataLoader
import torch
from sklearn.metrics import accuracy_score, precision_recall_fscore_support


# Define  metrics function
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = logits.argmax(axis=1)  # Get predicted class indices
    precision, recall, f1, _ = precision_recall_fscore_support(labels, predictions, average="weighted")
    accuracy = accuracy_score(labels, predictions)
    return {
        "accuracy": accuracy,
        "precision": precision,
        "recall": recall,
        "f1": f1,
    }
#  handle batching on CPU first then move to GPU
def collate_fn(batch):
    # Collect all elements into lists (CPU memory)
    input_ids = [x['input_ids'] for x in batch]
    attention_mask = [x['attention_mask'] for x in batch]
    labels = [x['label'] for x in batch]

    # Stack the list of tensors into one tensor (on CPU first)
    input_ids = torch.stack(input_ids, dim=0)
    attention_mask = torch.stack(attention_mask, dim=0)
    labels = torch.tensor(labels, dtype=torch.long)  # Convert list to tensor (CPU)


    return {'input_ids': input_ids, 'attention_mask': attention_mask, 'labels': labels}




# Define the training arguments
training_args = TrainingArguments(
    output_dir="./results",               # Output directory
    num_train_epochs=7,                   # Number of training epochs
    per_device_train_batch_size=32,       # Batch size for training
    per_device_eval_batch_size=32,        # Batch size for evaluation
    warmup_steps=500,                     # Number of warmup steps
    weight_decay=0.05,                    # Strength of weight decay
    logging_dir="./logs",                 # Directory for logs
    logging_steps=10,                     # Log every 10 steps
    evaluation_strategy="steps",          # Evaluate during training
    save_steps=500,                       # Save checkpoints every 500 steps
    save_total_limit=2,                   # Limit the number of saved checkpoints
    learning_rate=2e-5,                   # Learning rate
    fp16=True,                            # Mixed precision for faster training
    load_best_model_at_end=True,          # Load best model at the end
    report_to=["none"],                   # Disable wandb
)

# Freeze the first 8 layers and the embedding layer
for name, param in model.named_parameters():
    # Check if the parameter belongs to the encoder layers
    if name.startswith("bert.encoder.layer"):  # Adjust "bert" if the model name differs
        layer_number = int(name.split(".")[3])  # Extract the layer number
        if layer_number < 8:  # Freeze the first 8 layers
            param.requires_grad = False
    # Check if the parameter belongs to the embedding layer
    elif name.startswith("bert.embeddings"):  # Adjust "bert" if the model name differs
        param.requires_grad = False


for name, param in model.named_parameters():
    print(f"{name} - Frozen: {not param.requires_grad}")
# Define trainer
trainer = Trainer(
    model=model,                         # The pre-trained model
    args=training_args,                  # Training arguments
    train_dataset=train_data,            # Training dataset
    eval_dataset=test_data,              # Evaluation dataset
    tokenizer=tokenizer,                 # Tokenizer
    data_collator=collate_fn,            # Custom collate function
    compute_metrics=compute_metrics
)


trainer.train()


# Unfreeze all layers for fine-tuning
for param in model.parameters():
    param.requires_grad = True


training_args.learning_rate = 1e-6
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_data,
    eval_dataset=test_data,
    tokenizer=tokenizer,
    data_collator=collate_fn,
    compute_metrics=compute_metrics
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1575: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
<ipython-input-12-1354c020bd31>:70: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


bert.embeddings.word_embeddings.weight - Frozen: True
bert.embeddings.position_embeddings.weight - Frozen: True
bert.embeddings.token_type_embeddings.weight - Frozen: True
bert.embeddings.LayerNorm.weight - Frozen: True
bert.embeddings.LayerNorm.bias - Frozen: True
bert.encoder.layer.0.attention.self.query.weight - Frozen: True
bert.encoder.layer.0.attention.self.query.bias - Frozen: True
bert.encoder.layer.0.attention.self.key.weight - Frozen: True
bert.encoder.layer.0.attention.self.key.bias - Frozen: True
bert.encoder.layer.0.attention.self.value.weight - Frozen: True
bert.encoder.layer.0.attention.self.value.bias - Frozen: True
bert.encoder.layer.0.attention.output.dense.weight - Frozen: True
bert.encoder.layer.0.attention.output.dense.bias - Frozen: True
bert.encoder.layer.0.attention.output.LayerNorm.weight - Frozen: True
bert.encoder.layer.0.attention.output.LayerNorm.bias - Frozen: True
bert.encoder.layer.0.intermediate.dense.weight - Frozen: True
bert.encoder.layer.0.intermedi

Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
10,1.635500,1.648195,0.211812,0.125678,0.211812,0.101375
20,1.638100,1.644050,0.217199,0.139054,0.217199,0.106762
30,1.625300,1.636870,0.219315,0.153110,0.219315,0.106040
40,1.637700,1.626351,0.228549,0.166037,0.228549,0.118715
50,1.616400,1.611272,0.232012,0.361490,0.232012,0.123279
60,1.615000,1.588583,0.282416,0.392832,0.282416,0.207599
70,1.579100,1.557546,0.353982,0.358338,0.353982,0.325195
80,1.545800,1.524651,0.329742,0.369994,0.329742,0.282305
90,1.528300,1.499360,0.316468,0.355715,0.316468,0.254909
100,1.509900,1.480356,0.320893,0.418230,0.320893,0.259595


/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.10/dist-packages/sklearn/m

Step,Training Loss,Validation Loss,Accuracy,Precision,Recall,F1
10,0.242300,0.550387,0.803578,0.803321,0.803578,0.802719
20,0.258300,0.550388,0.803578,0.803303,0.803578,0.802735
30,0.296900,0.550436,0.803386,0.802994,0.803386,0.802575
40,0.283100,0.550623,0.803001,0.802579,0.803001,0.802171
50,0.261700,0.550834,0.802809,0.802328,0.802809,0.801966
60,0.334200,0.550617,0.803001,0.802617,0.803001,0.802214
70,0.264800,0.550407,0.803386,0.802945,0.803386,0.802635
80,0.242600,0.549959,0.803771,0.803582,0.803771,0.803076
90,0.285500,0.549826,0.803578,0.803813,0.803578,0.803197
100,0.251600,0.549761,0.803386,0.803659,0.803386,0.802989


Could not locate the best model at ./results/checkpoint-430/pytorch_model.bin, if you are running a distributed training on multiple nodes, you should activate `--save_on_each_node`.


TrainOutput(global_step=4550, training_loss=0.19639679211836594, metrics={'train_runtime': 3198.2033, 'train_samples_per_second': 45.464, 'train_steps_per_second': 1.423, 'total_flos': 3.825843038683546e+16, 'train_loss': 0.19639679211836594, 'epoch': 7.0})

In [ ]:


results = trainer.evaluate()
print("Evaluation results:", results)

Evaluation results: {'eval_loss': 0.5939663648605347, 'eval_accuracy': 0.8056944978838014, 'eval_precision': 0.8053234859297075, 'eval_recall': 0.8056944978838014, 'eval_f1': 0.8049755978807892, 'eval_runtime': 5.6434, 'eval_samples_per_second': 921.071, 'eval_steps_per_second': 28.883, 'epoch': 7.0}


In [ ]:

results = trainer.evaluate()

# Print the metrics
print("Evaluation results:")
print(results)

Evaluation results:
{'eval_loss': 0.5939663648605347, 'eval_accuracy': 0.8056944978838014, 'eval_precision': 0.8053234859297075, 'eval_recall': 0.8056944978838014, 'eval_f1': 0.8049755978807892, 'eval_runtime': 5.8927, 'eval_samples_per_second': 882.109, 'eval_steps_per_second': 27.661, 'epoch': 7.0}


In [ ]:


model.save_pretrained("./fine_tuned_marbert")


tokenizer.save_pretrained("./fine_tuned_marbert")

('./fine_tuned_marbert/tokenizer_config.json',
 './fine_tuned_marbert/special_tokens_map.json',
 './fine_tuned_marbert/vocab.txt',
 './fine_tuned_marbert/added_tokens.json',
 './fine_tuned_marbert/tokenizer.json')

In [ ]:

# Copy the model to Google Drive
!cp -r fine_tuned_marbert /content/drive/MyDrive/fine_tuned_marbert